# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
import datetime

In [4]:
import api_token  # handle iexcloud API tokens

env = "cloud" # "sandbox"  # 
BASE_URL, IEX_CLOUD_API_TOKEN = api_token.get_base_url(env=env), api_token.get_api_token(env=env)

In [5]:
TOP_N = 50
portfolio_size = 1000000

In [6]:
BASE_URL, IEX_CLOUD_API_TOKEN 

('https://cloud.iexapis.com', 'pk_34b4dac7dd61426cb38d3b14ffd5473d')

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [11]:
file_csv = "sector-basic-materials.csv"
df = pd.read_csv(file_csv, 
                 names=["Ticker", "Company"], 
                 skiprows=[0])

In [12]:
stocks = df["Ticker"].values.tolist()

In [13]:
",".join(stocks)

'ASIX,AEM,APD,AGI,ALB,AA,AXU,AAU,ACH,AVD,USAS,AMRS,AU,RKDA,MT,ASH,AUG,AVTR,AVNT,ASM,AXTA,BTG,BCPC,GOLD,BHP,BBL,BIOX,BCC,BAK,CBT,CMCL,CE,CPAC,CX,CENX,CF,CCF,CGA,CHNR,CINR,CLW,CLF,CDE,CSCW,CMC,SID,BVN,CMP,CSTM,CMT,CTVA,KOR,CRH,UAN,UFS,DOW,DRD,DD,EXP,EMN,ECL,EGO,ESI,EMX,EXK,EVA,EQX,EXN,FOE,GSM,AG,FSI,FMC,FSM,FNV,FCX,FRD,FF,GAU,GATO,GCP,GMO,GGB,GEVO,AUMN,GSS,GFI,GORO,GSV,GPL,GPRE,SIM,GURE,HMY,HWKN,FUL,HL,HBM,HDSN,HHT,HUN,HYMC,IAG,IKNX,NGVT,IOSP,ITRG,IFF,THM,IPI,ICL,ITP,JHX,JCTCF,KALU,KGC,KL,KOP,KRA,KRO,LIN,LAC,LTHM,LOMA,LOOP,LXU,LYB,MAG,MBII,MLM,MTRN,MMX,MUX,MDU,MTL,MERC,MSB,MTA,MEOH,MTX,MKD,MP,NP,NGD,NEU,NEM,NEXA,OSB,NAK,NTIC,NG,NUE,NTR,ODC,OLN,ZEUS,SEED,OEC,OR,OSN,PAAS,PZG,GLT,PLL,PLG,PLM,PKX,PPG,PQG,PVG,KWR,METC,RYAM,RS,RFP,RETO,RIO,RGLD,RPM,SAND,SCHN,SWM,SA,SNES,SXT,SBSW,SMTS,SVM,SILV,SMID,SQM,XPL,SCCO,SPPP,SSRM,STLD,SCL,SUM,SXC,SUZ,SYNL,TRX,TGB,TECK,TX,CC,MOS,SMG,SHW,TMST,TREC,TG,TMQ,TSE,TROX,TRQ,UAMY,USLM,X,UNVR,UFPI,USAP,USCR,USAU,VALE,VHI,VEDL,VNTR,VRS,VGZ,VMC,HCC,WDFC,WRN,WLK,WLKP

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [7]:
symbol = 'AEM'
api_url = f'{BASE_URL}/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Agnico Eagle Mines Ltd',
 'marketcap': 17385561388,
 'week52high': 86,
 'week52low': 35.96,
 'week52change': 0.2530090951267978,
 'sharesOutstanding': 242916884,
 'float': 0,
 'avg10Volume': 1058997,
 'avg30Volume': 1228910,
 'day200MovingAvg': 73.37,
 'day50MovingAvg': 71.57,
 'employees': 0,
 'ttmEPS': 2.64,
 'ttmDividendRate': 1.1222763960253324,
 'dividendYield': 0.01568082151774951,
 'nextDividendDate': '',
 'exDividendDate': '2020-11-24',
 'nextEarningsDate': '',
 'peRatio': 27.24646620415775,
 'beta': 0.36283746203704026,
 'maxChangePercent': 1.0546549115640005,
 'year5ChangePercent': 1.8988500998416304,
 'year2ChangePercent': 0.8180986808178776,
 'year1ChangePercent': 0.2530090951267978,
 'ytdChangePercent': 0.17918233253040627,
 'month6ChangePercent': 0.21877703379266644,
 'month3ChangePercent': -0.10853635295832165,
 'month1ChangePercent': 0.0640085037426874,
 'day30ChangePercent': 0.0640085037426874,
 'day5ChangePercent': 0.02009692132269092}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [8]:
data['year1ChangePercent']

0.2530090951267978

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [14]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks, 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [13]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,ATVI,83.72,0.440064,N/A
1,ADBE,476.04,0.568678,N/A
2,AMD,92.24,1.162495,N/A
3,ALXN,124.92,0.077344,N/A
4,ALGN,509.53,0.914253,N/A
...,...,...,...,...
98,WBA,42.66,-0.266520,N/A
99,WDAY,219.26,0.382707,N/A
100,XEL,68.00,0.091229,N/A
101,XLNX,145.21,0.520033,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [14]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:(TOP_N+1)]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MRNA,156.950,7.685771,N/A
1,TSLA,611.630,7.521929,N/A
2,ZM,408.100,5.547143,N/A
3,PDD,148.560,2.963366,N/A
4,DOCU,226.390,2.181137,N/A
5,MELI,1618.080,1.790635,N/A
6,JD,83.520,1.512043,N/A
7,NVDA,544.380,1.332223,N/A
8,AMD,92.240,1.162495,N/A
9,PYPL,222.680,1.058369,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [15]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

# portfolio_input()
# print(portfolio_size)

In [16]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/home/wengong/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MRNA,156.950,7.685771,124
1,TSLA,611.630,7.521929,32
2,ZM,408.100,5.547143,48
3,PDD,148.560,2.963366,131
4,DOCU,226.390,2.181137,86
5,MELI,1618.080,1.790635,12
6,JD,83.520,1.512043,234
7,NVDA,544.380,1.332223,36
8,AMD,92.240,1.162495,212
9,PYPL,222.680,1.058369,88


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [15]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'{BASE_URL}/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [17]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ASIX,20.10,N/A,-0.00633528,N/A,0.769965,N/A,0.596711,N/A,0.184088,N/A,N/A
1,AEM,71.70,N/A,0.253009,N/A,0.218777,N/A,-0.108536,N/A,0.0640085,N/A,N/A
2,APD,264.49,N/A,0.172128,N/A,0.156274,N/A,-0.084599,N/A,-0.00343833,N/A,N/A
3,AGI,9.19,N/A,0.792497,N/A,0.162075,N/A,-0.0178695,N/A,0.0644419,N/A,N/A
4,ALB,142.02,N/A,1.0075,N/A,0.784641,N/A,0.421766,N/A,0.0890308,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
237,WPM,42.59,N/A,0.560624,N/A,0.00643466,N/A,-0.177426,N/A,0.0195532,N/A,N/A
238,GRA,55.88,N/A,-0.195364,N/A,0.113217,N/A,0.234444,N/A,0.00436522,N/A,N/A
239,AUY,5.70,N/A,0.644689,N/A,0.130113,N/A,-0.0739375,N/A,0.0620301,N/A,N/A
240,YTEN,5.85,N/A,-0.110458,N/A,-0.12481,N/A,-0.247382,N/A,-0.123476,N/A,N/A


In [19]:
hqm_dataframe[hqm_dataframe[["One-Year Price Return"]].isnull()]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
hqm_dataframe[hqm_dataframe["One-Year Price Return"].isnull()]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


In [29]:
hqm_dataframe[hqm_dataframe["Six-Month Price Return"].isnull()]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


In [30]:
hqm_dataframe[hqm_dataframe["Three-Month Price Return"].isnull()]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


In [27]:
hqm_dataframe[hqm_dataframe["One-Month Price Return"].isnull()]

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score


In [26]:
hqm_dataframe = hqm_dataframe.dropna(how='any', 
                                     subset=["One-Month Price Return", "Three-Month Price Return", 
                                             "Six-Month Price Return", "One-Year Price Return"])

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [31]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# # Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

/home/wengong/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ASIX,20.10,N/A,-0.00633528,0.281513,0.769965,0.781513,0.596711,0.903361,0.184088,0.747899,N/A
1,AEM,71.70,N/A,0.253009,0.567227,0.218777,0.348739,-0.108536,0.121849,0.0640085,0.441176,N/A
2,APD,264.49,N/A,0.172128,0.504202,0.156274,0.268908,-0.084599,0.168067,-0.00343833,0.163866,N/A
3,AGI,9.19,N/A,0.792497,0.852941,0.162075,0.285714,-0.0178695,0.268908,0.0644419,0.445378,N/A
4,ALB,142.02,N/A,1.0075,0.894958,0.784641,0.794118,0.421766,0.819328,0.0890308,0.529412,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
237,WPM,42.59,N/A,0.560624,0.756303,0.00643466,0.142857,-0.177426,0.0714286,0.0195532,0.256303,N/A
238,GRA,55.88,N/A,-0.195364,0.105042,0.113217,0.231092,0.234444,0.663866,0.00436522,0.184874,N/A
239,AUY,5.70,N/A,0.644689,0.810924,0.130113,0.247899,-0.0739375,0.189076,0.0620301,0.428571,N/A
240,YTEN,5.85,N/A,-0.110458,0.180672,-0.12481,0.0546218,-0.247382,0.0252101,-0.123476,0.0294118,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [32]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

/home/wengong/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [33]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:(TOP_N+1)]

<ipython-input-33-d8b1b5c16398>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace=True)


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [34]:
# portfolio_input()

In [35]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)

In [36]:
position_size

19607.843137254902

In [37]:
hqm_dataframe.reset_index(drop = True, inplace = True)

In [38]:
len(hqm_dataframe['Ticker'])

51

In [39]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,PLG,5.1300,N/A,2.76515,0.97479,2.47552,0.983193,1.19912,0.978992,1.35545,1,0.984244
1,MP,35.7200,N/A,2.251,0.966387,2.251,0.978992,1.16013,0.97479,0.940896,0.995798,0.978992
2,GSM,1.6100,N/A,0.823575,0.87395,1.91228,0.966387,1.15584,0.970588,0.781116,0.983193,0.948529
3,SID,5.9700,N/A,0.767164,0.844538,1.81905,0.962185,0.940984,0.957983,0.6,0.97479,0.934874
4,IPI,16.0800,N/A,4.98127,0.987395,12.887,1,0.518061,0.865546,0.261453,0.882353,0.933824
5,CLF,13.8500,N/A,0.676158,0.819328,1.40714,0.936975,0.939568,0.953782,0.460455,0.957983,0.917017
6,CMT,13.9900,N/A,3.43464,0.978992,2.0426,0.970588,0.380468,0.773109,0.293613,0.911765,0.908613
7,SMID,9.2500,N/A,3.59799,0.983193,3.59799,0.987395,3.59799,1,0.126847,0.642857,0.903361
8,SEED,13.5000,N/A,1.9732,0.953782,0.803641,0.806723,0.432851,0.827731,0.391174,0.941176,0.882353
9,MTA,11.3600,N/A,222.846,1,1.02787,0.865546,0.382423,0.777311,0.258378,0.878151,0.880252


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [43]:
for i in range(len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])

/home/wengong/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [44]:
filename=file_csv.replace(".csv", "")

In [45]:
today = datetime.datetime.now()
xlsx_file = f"{filename}_{today.strftime('%Y-%m-%d')}.xlsx"

In [46]:
xlsx_file

'sector-basic-materials_2020-12-21.xlsx'

In [47]:
writer = pd.ExcelWriter(xlsx_file, engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [48]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [49]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 10, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [50]:
writer.save()